# Cut corresponding segments across perforamnces
audio, timing csv, maybe onto phys. 

In [9]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from IPython.display import Audio
import mir_eval.sonify
from pydub import AudioSegment

from scipy.signal import butter, filtfilt, argrelextrema
from scipy import interpolate
from scipy.interpolate import interp1d

In [10]:
%load_ext autoreload
%autoreload 1
%aimport qex
%aimport al
%aimport eq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
 %reload_ext autoreload

In [12]:
# audio file locations in
datadir = '../Data2024/'
audio_loc = '../Data2024/Audio/Full_Zero/'
out_loc = '../Data2024/Audio/Piece_wise/'
# audio file locations out

Piece_timings  = pd.read_csv(datadir + 'Timing/Lydo2024_Music_events_red.csv')
Piece_timings

,Event,Related,Event_Type,Full_Title,Composer,Performer,Screen,RD,C1,C2,...,C6,C7,RD.1,C1.1,C2.1,C3.1,C4.1,C5.1,C6.1,C7.1
0,Tuning,Tune,Music,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000,...,0.000000,0.000000,49.000,38.000,36.810,41.300,38.400,45.200,36.500,37.900
1,Pause,Tune,Silence,NaN,NaN,NaN,NaN,49.000000,38.000000,36.810,...,36.500000,37.900000,540.203,7.320,5.058,4.713,4.079,2.832,5.360,5.492
2,Marsj,Trad,Music,Gammel Jegermarsj,arr. Frederik Schöildberg (?),NaN,NaN,589.203333,45.320000,41.868,...,41.860000,43.392000,17.73,17.76666667,17.912,18.24,18.2115,17.928,18.04133333,17.68533333
3,MC,Trad,Speech,NaN,NaN,NaN,NaN,606.933333,63.086667,59.780,...,59.901333,61.077333,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!
4,20thCFox,Newm,Music,Fanfar (20th Cent. Fox),Alfred Newman,NaN,NaN,669.048000,138.070000,159.230,...,137.330000,158.080000,-669.048,-138.07,22.114,-143.038,22.9035,22.82933333,20.91666667,23.668
5,Applause,Newm,Applause,NaN,NaN,NaN,NaN,NaN,NaN,181.344,...,158.246667,181.748000,691.5333333,162.7133333,0.182,165.1113333,3.8475,5.618666667,6.776666666,6.156
6,MC,Gri1,Speech,NaN,NaN,NaN,NaN,691.533333,162.713333,181.526,...,165.023333,187.904000,23.6,28.56666667,27.69,28.29866667,29.2635,26.128,30.01666667,26.992
7,Morgenstemning,Gri1,Music,"Peer Gynt Suite No. 1, Op. 46 i Morgenstemning",Edvard Grieg,NaN,Preikestolen,715.133333,191.280000,209.216,...,195.040000,214.896000,81.98666667,89.724,86.936,91.31266667,88.92,92.92,92.23333333,89.85066667
8,Applause,Gri1,Applause,NaN,NaN,NaN,NaN,797.120000,281.004000,296.152,...,287.273333,304.746667,11.712,14.392,9.496,6.958,8.0775,11.32,8.576666667,9.130666666
9,Ildfuglen,Str1,Music,L'Oiseau de feu i. Introduction,Igor Stravinsky,NaN,Preikestolen,808.832000,295.396000,305.648,...,295.850000,313.877333,102.208,102.82,99.18,97.12733333,102.0285,97.216,96.51333333,95.95733333


In [13]:
datafile_type = 'Audio'
concert = 'C3'
piece = 'Full'
audio_file = audio_loc + concert + '_Full_Zero.wav'
C_times = Piece_timings[['Event','Event_Type','Related']].copy()
C_times['Start'] =  Piece_timings[concert]
C_times = C_times.sort_values('Start')
C_times = C_times.loc[C_times['Start']>-15,:].reset_index(drop = True).copy()
C_times['Next'] = C_times.Start.shift(-1)
C_times['Dur'] = C_times['Next']-C_times['Start']
M_times =  C_times.loc[C_times['Event_Type'] == 'Music',:]
M_times

,Event,Event_Type,Related,Start,Next,Dur
0,Tuning,Music,Tune,0.000000,41.300000,41.300000
2,Marsj,Music,Trad,46.013333,64.253333,18.240000
4,20thCFox,Music,Newm,143.038000,165.111333,22.073333
6,Morgenstemning,Music,Gri1,193.410000,284.722667,91.312667
8,Ildfuglen,Music,Str1,291.680667,388.808000,97.127333
11,LaTraviata,Music,Verd,507.733333,731.868667,224.135333
14,Kjempervisesåtten,Music,Saev,830.704000,1082.041333,251.337333
17,Hedvig,Music,Will,1161.990667,1294.057333,132.066666
20,Elevbidrag,Music,LieV,1406.043333,1653.194667,247.151334
22,BellaCiao,Music,Izab,1681.689333,1952.332667,270.643334


In [8]:
# sound = AudioSegment.from_wav(audio_file)
#sound = AudioSegment.from_mp3(audio_loc)
# print(len(sound))

In [15]:
# buffer is small, 1 second
t_units = 1000  # pydub does things in milliseconds
buff = 1 # in seconds
tic = time.time()
for i in M_times.index:
    Peice_tag = M_times.loc[i,'Related']
    s_time = np.max([(M_times.loc[i,'Start']-buff) * t_units,0]) # Tuning starts at zero for these so 
    e_time = (M_times.loc[i,'Next']+buff) * t_units
    p_audio = sound[s_time:e_time]
    out_audio = out_loc + concert + '_' + Peice_tag + '_1.mp3'
    with open(out_audio, 'wb') as out_f:
        p_audio.export(out_f, format='mp3')
    print([Peice_tag,np.round(time.time()-tic,2)])

['Tune', 1.24]
['Trad', 1.69]
['Newm', 2.26]
['Gri1', 4.36]
['Str1', 6.34]
['Verd', 11.15]
['Saev', 16.59]
['Will', 19.54]
['LieV', 24.64]
['Izab', 29.98]
['Gri2', 30.64]
['Pend', 31.72]
['Barb', 34.48]
['Viv1', 37.3]
['Ross', 45.97]


In [16]:
# OK now do them all
datafile_type = 'Audio'
piece = 'Full'
# buffer is small, 1 second
t_units = 1000  # pydub does things in milliseconds
buff = 1 # in seconds
for concert in ['RD','C1','C2','C4','C5','C6','C7']: # 'C3 has already been processsed'
    audio_file = audio_loc + concert + '_Full_Zero.wav'
    C_times = Piece_timings[['Event','Event_Type','Related']].copy()
    C_times['Start'] =  Piece_timings[concert]
    C_times = C_times.sort_values('Start')
    C_times = C_times.loc[C_times['Start']>-15,:].reset_index(drop = True).copy()
    C_times['Next'] = C_times.Start.shift(-1)
    M_times =  C_times.loc[C_times['Event_Type'] == 'Music',:]
    sound = AudioSegment.from_wav(audio_file)
    #sound = AudioSegment.from_mp3(audio_loc)
    print([concert,len(sound)])

    tic = time.time()
    for i in M_times.index:
        Peice_tag = M_times.loc[i,'Related']
        s_time = np.max([(M_times.loc[i,'Start']-buff) * t_units,0]) # Tuning starts at zero for these so 
        e_time = (M_times.loc[i,'Next']+buff) * t_units
        p_audio = sound[s_time:e_time]
        out_audio = out_loc + concert + '_' + Peice_tag + '_1.mp3'
        with open(out_audio, 'wb') as out_f:
            p_audio.export(out_f, format='mp3')
        print([Peice_tag,np.round(time.time()-tic,2)])

['RD', 3752474]
['Tune', 1.25]
['Trad', 1.69]
['Newm', 2.2]
['Gri1', 3.93]
['Str1', 5.54]
['Verd', 12.23]
['Saev', 17.45]
['Will', 20.24]
['LieV', 25.22]
['Izab', 31.02]
['Gri2', 31.65]
['Pend', 32.89]
['Barb', 37.3]
['Viv1', 40.13]
['Ross', 48.13]
['C1', 3182218]
['Tune', 1.0]
['Trad', 1.45]
['Newm', 2.02]
['Gri1', 3.93]
['Str1', 5.95]
['Verd', 10.75]
['Saev', 16.2]
['Will', 19.13]
['LieV', 24.27]
['Izab', 29.89]
['Gri2', 30.55]
['Pend', 31.67]
['Barb', 34.26]
['Viv1', 37.25]
['Ross', 45.74]
['C2', 3214929]
['Tune', 1.0]
['Trad', 1.44]
['Newm', 1.97]
['Gri1', 3.87]
['Str1', 5.86]
['Verd', 10.64]
['Saev', 16.13]
['Will', 18.97]
['LieV', 23.91]
['Izab', 29.38]
['Gri2', 29.99]
['Pend', 31.13]
['Barb', 33.82]
['Viv1', 36.83]
['Ross', 44.89]
['C4', 3124022]
['Tune', 0.98]
['Trad', 1.43]
['Newm', 1.99]
['Gri1', 3.94]
['Str1', 5.9]
['Verd', 10.51]
['Saev', 15.81]
['Will', 18.61]
['LieV', 23.72]
['Izab', 29.1]
['Gri2', 29.75]
['Pend', 30.79]
['Barb', 33.38]
['Viv1', 36.28]
['Ross', 44.23]
['C

# matching phys cuts

In [14]:
data_dir='/Volumes/TaeTae/Stavangar/bic2024/'
sep = '/'
DATAtag='EQDATA'
df_datafiles = eq.eq_recordings(data_dir+concert+'/','EQDATA','/')
#df_datafiles.iloc[:5,:]

In [15]:
data_dir='/Volumes/TaeTae/Stavangar/bic2024'
Show_folders = {
    'RD':data_dir + '/RD/',
    'C1':data_dir + '/C1/',
    'C2':data_dir + '/C2/',
    'C3':data_dir + '/C3/',
    'C4':data_dir + '/C4/',
    'C5':data_dir + '/C5/',
    'C6':data_dir + '/C6/',
    'C7':data_dir + '/C7/'}
concerts = list(Show_folders.keys())
Piece_folders = {
    'RD':data_dir + '/Pieces' + '/RD/',
    'C1':data_dir + '/Pieces' + '/C1/',
    'C2':data_dir + '/Pieces' + '/C2/',
    'C3':data_dir + '/Pieces' + '/C3/',
    'C4':data_dir + '/Pieces' + '/C4/',
    'C5':data_dir + '/Pieces' + '/C5/',
    'C6':data_dir + '/Pieces' + '/C6/',
    'C7':data_dir + '/Pieces' + '/C7/'}
# Saev - Sævarud, Reic - Reich, Arba - Arban, Tcha - Tsjakovskij, Chri - Christophersen, Viva - Vivaldi-Vinje, Stra - Strauss 
Pieces = ['Trad', 'Newm', 'Gri1', 'Str1', 'Verd', 'Saev', 'Will',
       'LieV', 'Izab', 'Gri2', 'Pend', 'Barb', 'Viv1', 'Ross']
# Full_composers = {'Saev':'Sævarud', 'Reic':'Reich', 'Arba':'Arban', 'Tcha':'Tsjakovskij', 'Chri':'Christophersen', 'Viva':'Vivaldi-Vinje', 'Stra':'Strauss'}
datafile_types = ['EQDATA','EQCIBI','EQRESP','EQBACC','EQ2ECG']

In [30]:
concert = 'C3'
C_times = Piece_timings[['Event','Event_Type','Related']].copy()
C_times['Start'] =  Piece_timings[concert]
C_times = C_times.sort_values('Start')
C_times = C_times.loc[C_times['Start']>-15,:].reset_index(drop = True).copy()
C_times['Next'] = C_times.Start.shift(-1)
C_times['Dur'] = C_times['Next']-C_times['Start']
M_times =  C_times.loc[C_times['Event_Type'] == 'Music',:]
M_times['Related'].values

array(['Tune', 'Trad', 'Newm', 'Gri1', 'Str1', 'Verd', 'Saev', 'Will',
       'LieV', 'Izab', 'Gri2', 'Pend', 'Barb', 'Viv1', 'Ross'],
      dtype=object)

In [34]:
M_times.set_index(M_times['Related'],inplace = True,drop = True)
M_times

,Event,Event_Type,Related,Start,Next,Dur
Related,,,,,,
Tune,Tuning,Music,Tune,0.000000,41.300000,41.300000
Trad,Marsj,Music,Trad,46.013333,64.253333,18.240000
Newm,20thCFox,Music,Newm,143.038000,165.111333,22.073333
Gri1,Morgenstemning,Music,Gri1,193.410000,284.722667,91.312667
Str1,Ildfuglen,Music,Str1,291.680667,388.808000,97.127333
Verd,LaTraviata,Music,Verd,507.733333,731.868667,224.135333
Saev,Kjempervisesåtten,Music,Saev,830.704000,1082.041333,251.337333
Will,Hedvig,Music,Will,1161.990667,1294.057333,132.066666
LieV,Elevbidrag,Music,LieV,1406.043333,1653.194667,247.151334


In [35]:
buffer_s = 15 # seconds before and after each piece is included in every excerpt

In [37]:
# across concerts
toc = time.time()
for concert in concerts:
    tic = time.time()
    data_loc = Show_folders[concert]
    files = os.listdir(data_loc)
    files.sort()
    C_times = Piece_timings[['Event','Event_Type','Related']].copy()
    C_times['Start'] =  Piece_timings[concert]
    C_times = C_times.sort_values('Start')
    C_times = C_times.loc[C_times['Start']>-15,:].reset_index(drop = True).copy()
    C_times['Next'] = C_times.Start.shift(-1)
    C_times['Dur'] = C_times['Next']-C_times['Start']
    M_times =  C_times.loc[C_times['Event_Type'] == 'Music',:]
    M_times.set_index(M_times['Related'],inplace = True,drop = True)


    # across datafile types
    for dft in datafile_types:
        sig_files = {}
        for f in files: 
            if f.endswith(dft+'.csv'): 
                partID = f.split('_')[2]
                sig_files[partID] = data_loc+f
        participants = list(sig_files.keys())

        # across participants
        for partID in participants:
            V = pd.read_csv(sig_files[partID])
            
            # across pieces
            for pieceID in Pieces:
                pZero = M_times.loc[pieceID,'Start']
                pEnd = M_times.loc[pieceID,'Next']


                X = V.loc[V['c_sTime']>pZero - buffer_s,:].copy()
                XX = X.loc[X['c_sTime']<pEnd + buffer_s,:].copy().reset_index(drop=True)
                cols = XX.columns
                signal_segment = pd.DataFrame()
                signal_segment['p_sTime'] = np.round(XX['c_sTime']-pZero,3)
                signal_segment.loc[:,cols] = XX

                filename = '_'.join([concert,pieceID,partID,dft]) + '.csv'
                filepath = Piece_folders[concert]+'/'.join([pieceID,dft,''])

                # check if the folder exists for project and set up locations
                if not os.path.isdir(Piece_folders[concert] + pieceID):
                    os.mkdir(Piece_folders[concert] + pieceID)
                if not os.path.isdir(Piece_folders[concert] + pieceID + '/' + dft):
                    os.mkdir(Piece_folders[concert] + pieceID + '/' + dft)

                signal_segment.to_csv(filepath + filename)
                #os.listdir(filepath)
                
        print(' '.join([concert,dft,str(np.round(time.time() - tic,3)),str(np.round(time.time() - toc,3))]))


RD EQDATA 7.291 7.292
RD EQCIBI 16.197 16.197
RD EQRESP 34.147 34.148
RD EQBACC 214.112 214.113
RD EQ2ECG 406.518 406.519
C1 EQDATA 7.794 414.313
C1 EQCIBI 18.111 424.63
C1 EQRESP 52.944 459.463
C1 EQBACC 340.505 747.024
C1 EQ2ECG 635.505 1042.025
C2 EQDATA 8.297 1050.322
C2 EQCIBI 18.367 1060.392
C2 EQRESP 53.99 1096.015
C2 EQBACC 346.812 1388.837
C2 EQ2ECG 663.847 1705.872
C3 EQDATA 7.17 1713.043
C3 EQCIBI 15.118 1720.99
C3 EQRESP 35.2 1741.073
C3 EQBACC 214.386 1920.258
C3 EQ2ECG 410.572 2116.445
C4 EQDATA 8.189 2124.635
C4 EQCIBI 16.816 2133.261
C4 EQRESP 37.719 2154.164
C4 EQBACC 229.598 2346.043
C4 EQ2ECG 434.376 2550.821
C5 EQDATA 17.412 2568.233
C5 EQCIBI 37.076 2587.897
C5 EQRESP 64.228 2615.049
C5 EQBACC 327.77 2878.592
C5 EQ2ECG 600.069 3150.89
C6 EQDATA 8.834 3159.724
C6 EQCIBI 16.035 3166.926
C6 EQRESP 33.341 3184.231
C6 EQBACC 206.709 3357.599
C6 EQ2ECG 394.902 3545.792
C7 EQDATA 9.549 3555.342
C7 EQCIBI 23.879 3569.672
C7 EQRESP 47.232 3593.025
C7 EQBACC 280.265 3826.058

# Clean out some files that should have been removed before

In [21]:
cutout = 'LS410'
for concert in ['C3','C4']:
    projectpath = Piece_folders[concert]
    file_locs = []
    for root, dirs, files in os.walk(projectpath):
        for file in files:
            if(file.lower().endswith(".csv")):
                if(not file.lower().startswith("._")):
                    if cutout in file:
                        #print(os.path.join(root,file))
                        os.remove(os.path.join(root,file)) 
#                     file_locs.append(os.path.join(root,file))

In [22]:
cutout = 'WW507'
for concert in ['C5','C6']:
    projectpath = Piece_folders[concert]
    file_locs = []
    for root, dirs, files in os.walk(projectpath):
        for file in files:
            if(file.lower().endswith(".csv")):
                if(not file.lower().startswith("._")):
                    if cutout in file:
                        #print(os.path.join(root,file))
                        os.remove(os.path.join(root,file)) 
#                     file_locs.append(os.path.join(root,file))